In [2]:
import numpy as np
import pandas as pd

In [3]:
wine = pd.read_csv('http://bit.ly/wine-date')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
from sklearn.model_selection import train_test_split

data_input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
data_target = wine[['class']].to_numpy()

# Training 80%, Test 20%
train_input, test_input, train_target, test_target =\
    train_test_split(data_input, data_target, test_size=0.2, random_state=1234)

print(train_input.shape, test_input.shape)
print(train_target.shape, test_target.shape)

(5197, 3) (1300, 3)
(5197, 1) (1300, 1)


In [5]:
# Train 64%, Validate 16%, Test: 20%
train_input, val_input, train_target, val_target =\
    train_test_split(train_input, train_target, test_size=0.2, random_state=1234)

print(train_input.shape, val_input.shape, test_input.shape)
print(train_target.shape, val_target.shape, test_target.shape)

(4157, 3) (1040, 3) (1300, 3)
(4157, 1) (1040, 1) (1300, 1)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=1234)
dt.fit(train_input, train_target)

print('R Score (Train Set):', dt.score(train_input, train_target).round(4))
print('R Score (Val Set):', dt.score(val_input, val_target).round(4))

R Score (Train Set): 0.9983
R Score (Val Set): 0.8702


### k-fold Cross Validation

In [7]:
from sklearn.model_selection import cross_validate

result = cross_validate(dt, train_input, train_target, cv=10)
result_df = pd.DataFrame(result)
result_df

,fit_time,score_time,test_score
0,0.004986,0.000998,0.858173
1,0.006019,0.000000,0.843750
2,0.004987,0.000000,0.826923
3,0.005987,0.000000,0.865385
4,0.005981,0.000000,0.865385
5,0.004987,0.000000,0.848558
6,0.005984,0.000000,0.865385
7,0.005983,0.000000,0.879518
8,0.004987,0.000997,0.843373
9,0.004988,0.000996,0.855422


In [12]:
print('Average Validate Score:', np.round(result_df['test_score'].mean(), 4))

Average Validate Score: 0.8552


### k-fold Cross Validation (Stratified)

In [13]:
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

result = cross_validate(dt, train_input, train_target, cv=splitter)
result_df = pd.DataFrame(result)
result_df

,fit_time,score_time,test_score
0,0.005985,0.000998,0.858173
1,0.005982,0.000000,0.850962
2,0.004985,0.000998,0.870192
3,0.004987,0.000997,0.850962
4,0.005984,0.000998,0.836538
5,0.004985,0.000998,0.855769
6,0.004987,0.000998,0.858173
7,0.005983,0.000000,0.850602
8,0.004987,0.000000,0.836145
9,0.004986,0.000998,0.867470


In [14]:
print('Average Validate Score:', np.round(result_df['test_score'].mean(), 4))

Average Validate Score: 0.8535


### Grid Search

In [15]:
from sklearn.model_selection import GridSearchCV

params_dict = {
    'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}

dt = DecisionTreeClassifier(random_state=1234)

gs = GridSearchCV(dt, params_dict, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=1234), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
dt = gs.best_estimator_
print(dt)

DecisionTreeClassifier(min_impurity_decrease=0.0003, random_state=1234)


In [20]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_impurity_decrease,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008378,0.001196,0.001396,0.000490,0.0001,{'min_impurity_decrease': 0.0001},0.853365,0.850962,0.855596,0.862816,0.851986,0.854945,0.004231,5
1,0.007381,0.000802,0.001395,0.000486,0.0002,{'min_impurity_decrease': 0.0002},0.860577,0.858173,0.872443,0.868833,0.862816,0.864568,0.005293,2
2,0.007178,0.000745,0.001201,0.000398,0.0003,{'min_impurity_decrease': 0.0003},0.861779,0.865385,0.873646,0.861613,0.872443,0.866973,0.005151,1
3,0.007579,0.000490,0.000998,0.000004,0.0004,{'min_impurity_decrease': 0.0004},0.859375,0.854567,0.867629,0.866426,0.859206,0.861441,0.004892,4
4,0.006779,0.000399,0.000599,0.000489,0.0005,{'min_impurity_decrease': 0.0005},0.853365,0.859375,0.882070,0.865223,0.854392,0.862885,0.010475,3


In [21]:
print(gs.cv_results_['mean_test_score'].round(4))

[0.8549 0.8646 0.867  0.8614 0.8629]


In [22]:
best_idx = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_idx])

{'min_impurity_decrease': 0.0003}


### Grid Search (complex version)

In [23]:
params_dict = {
    'min_impurity_decrease': np.arange(0.0001, 0.0010, 0.0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10)
}

dt = DecisionTreeClassifier(random_state=1234)

gs = GridSearchCV(dt, params_dict, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=1234), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [24]:
pd.DataFrame(gs.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_impurity_decrease,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006582,0.001738,0.001396,0.000489,5,0.0001,2,"{'max_depth': 5, 'min_impurity_decrease': 0.00...",0.853365,0.856971,0.861613,0.836342,0.854392,0.852537,0.008584,1322
1,0.009375,0.007794,0.000798,0.000747,5,0.0001,12,"{'max_depth': 5, 'min_impurity_decrease': 0.00...",0.853365,0.856971,0.861613,0.837545,0.854392,0.852777,0.008132,1307
2,0.005385,0.000489,0.000998,0.000001,5,0.0001,22,"{'max_depth': 5, 'min_impurity_decrease': 0.00...",0.853365,0.859375,0.861613,0.837545,0.854392,0.853258,0.008431,1237
3,0.004586,0.000489,0.000798,0.000399,5,0.0001,32,"{'max_depth': 5, 'min_impurity_decrease': 0.00...",0.853365,0.859375,0.861613,0.833935,0.854392,0.852536,0.009791,1323
4,0.006783,0.004608,0.000996,0.000631,5,0.0001,42,"{'max_depth': 5, 'min_impurity_decrease': 0.00...",0.855769,0.859375,0.861613,0.833935,0.854392,0.853017,0.009878,1289


In [25]:
print(gs.best_params_)

{'max_depth': 15, 'min_impurity_decrease': 0.00030000000000000003, 'min_samples_split': 2}


### Random Search

In [26]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([3, 8, 5, 0, 1, 1, 3, 5, 2, 1])

In [27]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97, 107,  98,  95, 101, 106,  89, 112, 104,  91], dtype=int64))

In [28]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.56168744, 0.37480126, 0.57504075, 0.97308525, 0.40499052,
       0.95311773, 0.63882419, 0.12471848, 0.3880794 , 0.14906296])

In [29]:
params_dict = {
    'min_impurity_decrease': uniform(0.0001, 0.0010),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25)
}

from sklearn.model_selection import RandomizedSearchCV

dt = DecisionTreeClassifier(random_state=1234)

gs = RandomizedSearchCV(dt, params_dict, n_iter=100, n_jobs=-1, random_state=1234)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=1234),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EAD71D75E0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EAD71D70D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EAD6FE3F40>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EAD6FE3EB0>},
                   random_state=1234)

In [30]:
print(gs.best_params_)

{'max_depth': 28, 'min_impurity_decrease': 0.0009714232550486981, 'min_samples_leaf': 6, 'min_samples_split': 15}


In [31]:
pd.DataFrame(gs.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_impurity_decrease,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005585,0.000798,0.001397,0.000489,35,0.000597664,22,14,"{'max_depth': 35, 'min_impurity_decrease': 0.0...",0.866587,0.853365,0.865223,0.850782,0.858002,0.858792,0.006266,45
1,0.005783,0.000747,0.001596,0.000799,40,0.000885359,16,19,"{'max_depth': 40, 'min_impurity_decrease': 0.0...",0.866587,0.854567,0.867629,0.855596,0.856799,0.860236,0.005665,11
2,0.005984,0.001093,0.001197,0.000398,43,0.000250637,12,14,"{'max_depth': 43, 'min_impurity_decrease': 0.0...",0.871394,0.841346,0.856799,0.845969,0.866426,0.856387,0.011497,89
3,0.006780,0.003116,0.001197,0.000398,48,0.000975933,17,7,"{'max_depth': 48, 'min_impurity_decrease': 0.0...",0.872596,0.854567,0.867629,0.849579,0.856799,0.860234,0.008544,16
4,0.005585,0.001739,0.001396,0.000796,36,0.000812702,23,17,"{'max_depth': 36, 'min_impurity_decrease': 0.0...",0.867788,0.855769,0.867629,0.845969,0.855596,0.858550,0.008277,55


In [32]:
print('Max Mean Test Score:', gs.cv_results_['mean_test_score'].max().round(4))

Max Mean Test Score: 0.8619


In [33]:
dt = gs.best_estimator_

print('R Score (Train Set):', dt.score(train_input, train_target).round(4))
print('R Score (Test Set):', dt.score(test_input, test_target).round(4))

R Score (Train Set): 0.8756
R Score (Test Set): 0.8646
